In [3]:
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!ngrok authtoken XXX

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import requests
from flask import Flask, render_template, request

import os
import threading

import json,urllib.request
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"


public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))


app.config["BASE_URL"] = public_url


app = Flask(__name__)

tree = ET.parse('bus02.xml')
root = tree.getroot()


getAll={"PlateNumb":[],"PositionLon":[],"PositionLat":[]}
PlateNumb = {"PlateNumb":[]}
PositionLon={"PositionLon":[]}
PositionLat={"PositionLat":[]}


Bus=[]
ansA=[]
bb=[]
Lon=[]
Lat=[]
@app.route('/', methods=('GET', 'POST'))
def index():

    if request.method == 'POST':

        dropdownval = request.form.get('Gcity')
        bb.append(dropdownval)

        for No in root.iter('{https://ptx.transportdata.tw/standard/schema/}BusA1Data'):
            ansA.append(No[0].text)

        for key, value in getAll.items():
            for i in value:
                if i == dropdownval:
                    print(value.index(i))

                    Lon.append(getAll['PositionLon'][value.index(i)])
                    Lat.append(getAll['PositionLat'][value.index(i)])
                    print(Lon,Lat)



        render_template('example.html',bb=bb,Bus=Bus,Lon=Lon,Lat=Lat)

    for No in root.iter('{https://ptx.transportdata.tw/standard/schema/}BusA1Data'):
        getAll['PlateNumb'].append(No[0].text)
        Bus.append(No[0].text)
    for No in root.iter('{https://ptx.transportdata.tw/standard/schema/}BusPosition'):
        getAll['PositionLon'].append(No[0].text)
    for No in root.iter('{https://ptx.transportdata.tw/standard/schema/}BusPosition'):
        getAll['PositionLat'].append(No[1].text)

    return render_template('example.html',aa='0', Lon=Lon,Lat=Lat,getAll=getAll,Bus=Bus,bb=bb)


threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://2550-34-125-203-156.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
